In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as ss
import scipy.fftpack as fft
import seaborn as sns
from sklearn.linear_model import LinearRegression
import geopandas
import cartopy
pd.options.display.max_columns = 999
pd.options.display.max_rows = 77
from shapely.geometry import Point
import shapely.ops as shops

%matplotlib inline

In [ ]:
def read_data(filename, usecols=[2, 5, 8, 19, 20], dtype={'Primary Type': str, 'Arrest': bool, 'Latitude' : float, 'Longitude' : float}, **kwargs):

    data = pd.read_csv(filename, usecols=usecols, dtype=dtype, quotechar='"', index_col='Date', **kwargs)
    data.index = pd.to_datetime(data.index, format='%m/%d/%Y %I:%M:%S %p')
    return data

In [ ]:
def add_counts(crime_df, crime_type=None):
    """Add the counts by function of hour, day, month, and year
    
    If crime_type is None, then all crimes. Otherwise, for crime_type only"""
    
    crime_count_dict = {}
    
    if crime_type is not None and str.lower(crime_type) != 'all':
        crime_df = crime_df[crime_df['Primary Type'] == str.upper(crime_type)]
    
    if len(crime_df) == 0:
        raise KeyError('Invalid crime type')
    
    crime_count_dict['hour'] = crime_df.index.hour.value_counts().astype(float)
    crime_count_dict['weekday'] = crime_df.index.weekday.value_counts().astype(float)
    crime_count_dict['day'] = crime_df.index.day.value_counts().astype(float)
    crime_count_dict['month'] = crime_df.index.month.value_counts().astype(float)
    crime_count_dict['year'] = crime_df.index.year.value_counts().astype(float)
    
    return crime_count_dict

In [ ]:
def plot_counts(counts, ax, weights=None, bar=True, do_err=False, **kwargs):
    """Provide bar position and heights for crime counts
    
    if weights is not None, weightsalize 
    If bar false, do a line plot
    """
    xs = counts.index
    vals = counts.values / counts.values.sum()
    
    if weights is not None:
        try:
            vals *= weights
        except TypeError:
            if weights.lower() == 'month':
                weights = 1. / np.array([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]) * 30. 
                #technically Feb is 28.25?
            elif weights.lower() == 'day':
                weights = np.ones(31)
                weights[30] = 12./7
                weights[29] = 12./11
                weights[28] = 12./11
            else:
                print(f'Unrecognized weights: {weights}')
                weights = np.ones(len(xs))
            assert len(weights) == len(xs)
            vals *= weights
    if bar:
        ax.bar(xs, vals, **kwargs)
    else:
        if do_err:
            ax.errorbar(np.sort(xs), vals[np.argsort(xs)], yerr=np.sqrt(counts.values)/counts.values.sum(), **kwargs)
        else:
            ax.plot(np.sort(xs), vals[np.argsort(xs)], **kwargs)

In [ ]:
def plot_crime_map(crime_df, geo_df, census_df, year, crime_type, cmap='Blues', annotate=False,
                  num_annotations=(3, 3), annot_colors=('black', 'black'), filename=None, **kwargs):
    """Plot the crime rate by neighborhood
    
    crime_df : the crime dataframe
    geo_df : the geopandas dataframe, for drawing the map
    census_df : the census dataframe
    year : either one year or a range of years, inclusively, (min_year, max_year)
    crime_type : the type of crime to consider
    cmap (default 'Blues') : the colormap to use
    annotate (default False) : whether to label the highest and/or lowest crime neighborhoods, using
        the remaining arguments as guide
    num_annotations (default (3, 3)) : 2-tuple with (num_low, num_high), the number of neighborhoods to
        label with the lowest and highest crime rates, respectively
    annot_colors (default ('black', 'black')) : 2-tuple with colors to use to plot low and high neighborhoods
    filename (default None) : save the figure to the path given by filename
    
    **kwargs
    vmax - maximum of colorbar
    """
    
    try:
        a = len(year)
    except TypeError:
        year = [year]
    
    if len(year) == 1:
        subview = crime_df.loc[crime_df['Year'] == year[0]]
    elif len(year) == 2:
        subview = crime_df.loc[(crime_df['Year'] >= year[0]) & (crime_df['Year'] <= year[1])]
    else:
        raise ValueError('Too many years!')
        
    neighborhood_counts = (subview[subview['Primary Type'] == crime_type.upper()]['Community Area']
                                                .value_counts(sort=False))
    neighborhood_counts = neighborhood_counts.reindex(index=geo_df['area_num_1'].astype(int), fill_value=0)
    neighborhood_rates = (1e5 * neighborhood_counts / census_df['Total Population'] / len(subview['Year']
                                                .value_counts()))

    cmap1 = plt.cm.ScalarMappable(cmap=cmap)
    if 'vmax' in kwargs:
        cmap1.set_clim(0, kwargs['vmax'])
    else:
        cmap1.set_clim(0, neighborhood_rates.max())

    ax = geo_df.plot(edgecolor='k', 
                    facecolor=cmap1.to_rgba(neighborhood_rates.loc[geo_df['area_num_1'].astype(int)]), 
                    figsize=(10, 8))
    
    fig = ax.get_figure()
    cax = fig.add_axes([0.88, 0.1, 0.025, 0.8])
    cbar = fig.colorbar(cmap1, cax=cax)
    if len(year) > 1:
        year_label = f'{year[0]} to {year[1]}'
    else:
        year_label = f'{year[0]}'
    cbar.set_label(f'{crime_type.title()} per 100k people per year, ' + year_label, 
                   rotation=270, fontsize=20, labelpad=20)
    ax.set_xlim([-88, -87.5])
    ax.set_ylim([41.6, 42.05])
    
    if annotate:
        # lowest crime rate not necessarily unique if there are multiple zeros
        annotate_inds = []
        annotate_inds += list(np.arange(num_annotations[0]))
        annotate_inds += list(np.arange(-num_annotations[1], 0))
        to_annotate = neighborhood_rates.sort_values().iloc[annotate_inds]
        if len(to_annotate) > 0:
            # location of neighborhood centers
            xys = np.array([[geo_df[geo_df['area_num_1'] == neighborhood].centroid.x.values[0], 
                   geo_df[geo_df['area_num_1'] == neighborhood].centroid.y.values[0]]
                   for neighborhood in to_annotate.index])
            second_leg = []
            for i, (x, y) in enumerate(xys):
    #             ax.text(x, y, geo_df[geo_df['area_num_1'] == to_annotate.index[i]]['community'].values[0].title())
                ax.scatter(x, y, marker=f'${to_annotate.index[i]}$', 
                                  color=annot_colors[0 if i<num_annotations[0] else 1], 
                                  s=50 if to_annotate.index[i] < 10 else 100)
                # this one is just for the legend, to make sure the marker is black and legible
                scat = ax.scatter([], [], marker=f'${to_annotate.index[i]}$', color='black',
                                  s=100 if to_annotate.index[i] < 10 else 200,
                                  label=geo_df[geo_df['area_num_1'] 
                                   == to_annotate.index[i]]['community'].values[0].title()
                          + f', {neighborhood_rates.loc[to_annotate.index[i]]:.1f}')
                if i == num_annotations[0] - 1:
                    leg = ax.legend(loc='center left', 
                                    title=f'Lowest Crime Area\nID Number, Name, {crime_type.title()}/100k',
                                   fontsize=12, title_fontsize=12)
                    ax.add_artist(leg)
                    continue
                if i >= num_annotations[0]:
                    second_leg.append(scat)
                if i == len(xys) - 1:
                    ax.legend(handles=second_leg, loc='lower left', 
                              title=f'Highest Crime Area\nID Number, Name, {crime_type.title()}/100k',
                             fontsize=12, title_fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])
    if filename is not None:
        fig.savefig(filename, bbox_inches='tight')

In [ ]:
nrows = 1e6
filename = '/home/elaad/Documents/Fun/DatAnalysis/datasets/Crimes_-_2001_to_present.csv'
crimedata = read_data(filename, nrows=nrows, usecols=None, dtype=None)
print(len(crimedata[crimedata.Longitude <= -90]))
crimedata = crimedata[crimedata.Longitude > -90] # getting rid of a few unexpected outliers beyond the city limits
crimedata.sort_index(inplace=True)
crimedata = crimedata[crimedata['Year'] <= 2019] # removing the handful of 2020 reports for clarity

In [ ]:
census_dat = pd.read_excel('/home/elaad/Documents/Fun/DatAnalysis/datasets/CCASF12010CMAP.xlsx',
                          index_col=1, header=1, skiprows=0)

# Overview

The dataset represents a collection of ${\sim}7$ million crimes reported in Chicago between 2001 and 2019 (I hae excluded a small slice of 2020 that is already incorporated). What are the most common crimes?

NB: With the exception of homicides, all data are from reported crimes. The disclaimer from the data website is "These crimes may be based upon preliminary information supplied to the Police Department by the reporting parties that have not been verified. The preliminary crime classifications may be changed at a later date based upon additional investigation and there is always the possibility of mechanical or human error." Therefore, the dataset may contain errors. However, I will assume for the sake of this analysis that most crimes are accurately reported and that no results depend sensitively on small changes to the dataset.

In [ ]:
crimedata['Primary Type'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10, 10)
crimedata['Primary Type'].value_counts().plot(kind='bar', ax=ax)
ax.set_yscale('log')
ax.set_ylabel('N', fontsize=18)
ax.tick_params(labelsize=14)

So, thefts are the most common crime, followed by battery. This is not terribly surprising. Of, say, thefts, what are the most common descriptors?

In [ ]:
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(10, 10)
crimedata[crimedata['Primary Type'] == 'THEFT']['Description'].value_counts().plot(kind='bar', ax=ax)
ax.set_yscale('log')

The most common theft description is \\$500 and under, followed by over \\$500; small amounts are more common than large. For Financial ID Theft, however, it is the other way around, with higher amounts more common.

That last "theft retail" appears to be an error; "retail theft" is a much more common descriptor.

# Crimes as a function of time and date

Some assumptions certainly go into the believability of these plots.
For example:
* Listed information---particularly dates and times---are accurate (the overabundance of crimes on the first of the month casts doubt on this assumption)

In [ ]:
crime_counts = {}
for crime_type in ['all', 'homicide', 'theft', 'battery']:
    crime_counts[crime_type] = add_counts(crimedata, crime_type)

In [ ]:
for time_interval in crime_counts['all']:
    fig, ax = plt.subplots(1, 1)
    fig.set_size_inches(7, 5)
    for crime_type, color in zip(crime_counts, [f'C{i}' for i in range(len(crime_counts))]):
        plot_counts(crime_counts[crime_type][time_interval], ax, weights=time_interval, bar=False, do_err=False,
                    label=crime_type.capitalize(), color=color,)# alpha=1./len(crime_counts),)
#                     hatch=hatch)
    ax.legend(loc='best', fontsize=14)
    ax.set_xlabel(time_interval.capitalize(), fontsize=18)
    ax.set_ylabel('N/N$_\mathrm{tot}$', fontsize=18)
    ax.set_ylim([0, ax.set_ylim()[-1] * 1.2])
    ax.tick_params(labelsize=12)
    if time_interval == 'year':
        ax.set_xticklabels([str(year) for year in np.arange(2001, 2021, 2)])
        ax.set_xticks(np.arange(2001, 2021, 2))
    if time_interval == 'weekday':
        ax.set_xticklabels(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                          rotation=25)
        ax.set_xticks(np.arange(7))

A few interesting things to note here. Crimes generally seem to be least frequent around 5 AM, while homicides bottom out around 7 AM. Homicides also seem to peak a bit later than most crimes. Thefts, on the other hand, peak in the afternoon (except for a single spike around noon, which I will assume for now comes from thefts around the lunch hour. To test this I should see which specific types of theft are contributing to the spike). All of these observations are consistent with a general idea that most crimes require people to be around, while homicides occur when people *aren't* around.

There aren't strong trends with day of the week, except for a weekday/weekend divide. Homicides are less common on weekdays and more common on weekends, while other crimes are generally more common during the week. Future work will require separating crimes geographically. How much of this trend comes from crimes being concentrated in business areas?

For day of the month, I have weighted the histograms so that long months have their extra days' counts proportionately increased. With the exception of the first and last day of the month, variations look like noise. I assume (for now) that the differences in the first/last days are reporting biases of some sort.

For month of year, I have also weighted months by their length, so I have upweighted February and downweighted long months.

Crime generally has gone down over the last 19 years, almost by a factor of 2. The decline may be leveling, however. Starting in 2015, as well, murders spiked significantly, though perhaps they are returning again to their previous levels.

# Where do crimes occur?

In [ ]:
geodf = geopandas.read_file('../datasets/Boundaries_Community_Areas/')
geodf['area_num_1'] = geodf['area_num_1'].astype(int)

In [ ]:
plot_crime_map(crimedata, geodf, census_dat, (2001, 2019), 'homicide', cmap='Blues', 
               annotate=True, num_annotations=(5, 5), annot_colors=('black', 'white'))

Looks like homicides are very inhomogenously distributed; a few neighborhoods on the South and West sides have the majority of homicides. Most chicago neighborhoods have very low incidence of homicide. Thefts, on the other hand, are another matter:

In [ ]:
plot_crime_map(crimedata, geodf, census_dat, (2001, 2019), 'theft', cmap='Blues', 
               annotate=True, num_annotations=(5, 5), annot_colors=('black', 'white'))

By a huge margin, the highest rate of theft is in the Loop, which is Chicago's most important commercial center. We can see in the list of descriptions below that, unlike what we saw above, the most common thefts are From Building and Retail Theft. This makes sense, given the nature of the neighborhood. We can also probably safely assume that such a commercial area is not only subject to more thefts, but is also more diligent in reporting them. Finally, the rate (per 100k people) is based on census data, but a commercial area like the Loop likely has far more people working there than residing there, which would inflate the rate.

In [ ]:
crimedata[(crimedata['Primary Type'] == 'THEFT') 
          & (crimedata['Community Area'] == 32)]['Description'].value_counts()

In [ ]:
# uncomment to make figures for every year
# for year in np.arange(2001, 2020, 1):
#     plot_crime_map(crimedata, geodf, census_dat, year, 'homicide', cmap='Blues', 
#                annotate=False, filename=f'figures/homicide_map_{year}.png', vmax=150)

# FFT - Not done

In [ ]:
# resampling crimes and homicides into daily counts
crime_days = (crimedata.index - pd.to_datetime('Jan 01 2001')).days.value_counts().sort_index()
# homicide_days = (crimedata[crimedata['Primary Type'] == 'HOMICIDE'].index - 
#                  pd.to_datetime('Jan 01 2001')).days.value_counts().sort_index()

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(crime_days/crime_days.sum())
ax.plot(homicide_days/homicide_days.sum()+0.00025)

In [ ]:
homicide_fft = fft.fft(homicide_days.values)
homicide_freq = fft.fftfreq(len(homicide_days))
crime_fft = fft.fft(crime_days.values)
crime_freq = fft.fftfreq(len(crime_days))

In [ ]:
plt.plot(homicide_freq, np.abs(homicide_fft))
plt.yscale('log')

Peak in one year period, as expected

In [ ]:
plt.figure(figsize=(10, 10))
# plt.plot(crime_freq, np.abs(crime_fft))
plt.plot(np.linspace(0, 1, len(crime_fft))**-1 / 365, np.abs(crime_fft))
plt.yscale('log')
# plt.loglog([], [])
plt.xlim([0, 4])
plt.axvline(1, color='red', ls='--')
plt.axvline(0.5, color='red', ls='--')

In [ ]:
plt.figure(figsize=(10, 10))
filt_crime_fft = crime_fft.copy()
filt_crime_fft[np.abs(filt_crime_fft) < 75000] = 0
plt.plot(np.linspace(0, len(crime_days)/365, len(crime_days)), crime_days)
plt.plot(np.linspace(0, len(crime_days)/365, len(crime_days)), fft.ifft(filt_crime_fft))

plt.plot(np.linspace(0, len(crime_days)/365, len(crime_days)), crime_days - fft.ifft(filt_crime_fft))

In [ ]:
def gaussian(x, mu, sigma):
    x = np.array(x)
    return 1. / np.sqrt((2 * np.pi * sigma)) * np.exp(-(x - mu)**2 / (2 * sigma))

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(crime_days)
plt.plot(np.convolve(crime_days, gaussian(np.arange(-20, 20, 1), 0, 7))) #gaussian smoothing with 1 week sigma